In [1]:
import pandas as pd
df = pd.read_csv('./data/target.csv', sep=',', header=None)
df.columns = ['latitude', 'longitude', 'year', 'month', 'depth', 'temperature']
df.insert(6,"interpolated_length",0)
print(df.tail())


        latitude  longitude  year  month  depth  temperature  \
432693    33.505    127.075  2019     12    200         -1.0   
432694    33.505    127.075  2019     12    250         -1.0   
432695    33.505    127.075  2019     12    300         -1.0   
432696    33.505    127.075  2019     12    400         -1.0   
432697    33.505    127.075  2019     12    500         -1.0   

        interpolated_length  
432693                    0  
432694                    0  
432695                    0  
432696                    0  
432697                    0  


In [2]:
csv_data = df.to_numpy()

ds = []
temp = []


for data in csv_data:
    if data[4] == 0:
        ds.append(temp)
        temp = []
    temp.append(data)
ds.append(temp)

del ds[0] # delete 0th [] list

In [3]:
len(ds) #30907 * 14row

30907

In [4]:
import numpy as np
def max_to_min_interpolate(min_idx, max_idx, data):
    print(max_idx)
    print(min_idx)
    depth_list = []
    for i in range(min_idx+1, max_idx):
        depth_list.append(data[i][4])
    result_list = np.interp(depth_list, [data[min_idx][4],data[max_idx][4]], [data[min_idx][5],data[max_idx][5]])
    
    r_idx = 0
    for i in range(min_idx+1, max_idx):
        data[i][5] = result_list[r_idx]
        r_idx += 1
    for i in range(14):
        data[i][6] = data[i][6] + max_idx - min_idx - 1

In [5]:
def check_and_interpolate(data, data_list):
    if data[0][5] == -1:  # if data's 0 meter temperature is 0
        return 
    else:
        max_idx = 13
        
        while max_idx > 0:
            if data[max_idx][5] >= 0:
                min_idx = max_idx - 1
                while min_idx >= 0:
                    if data[min_idx][5] >= 0:
                        if min_idx == max_idx - 1:
                            max_idx = min_idx
                            break
                        max_to_min_interpolate(min_idx,max_idx,data)
                        max_idx = min_idx
                        break
                    else:
                        min_idx -= 1
            else:
                max_idx -= 1
        data_list.append(data)
        return
            

In [6]:
interpolated_list = []
i = 0
for data in ds:
    check_and_interpolate(data,interpolated_list)
    if i%100 == 0:
        print("{}th data".format(i))
    i += 1

0th data
5
0
12
9
5
1
6
3
13
10
6
1
9
3
3
0
9
6
5
2
5
0
5
1
7
0
7
4
4
0
4
1
3
0
6
1
8
1
6
3
3
0
11
8
7
2
4
0
7
4
3
0
11
8
3
0
8
5
3
0
4
1
3
0
10
7
6
3
5
1
8
5
5
0
3
0
3
0
7
4
3
0
3
0
5
0
7
0
5
0
7
4
4
1
100th data
4
1
5
2
3
0
4
1
5
2
5
0
4
1
3
0
4
0
3
0
4
1
4
1
5
2
6
3
4
0
7
0
9
6
13
8
3
0
5
0
7
4
8
5
3
0
4
1
200th data
8
4
5
2
4
1
5
1
7
2
9
6
4
1
8
5
4
1
13
10
3
0
13
10
4
1
7
2
4
1
7
4
3
0
4
1
4
1
6
3
6
1
5
0
5
0
4
1
5
0
7
4
4
0
5
2
5
1
4
1
3
0
4
1
4
1
3
0
4
1
3
0
4
1
4
1
5
1
300th data
4
1
5
0
4
1
4
1
6
1
6
3
3
0
4
1
5
1
4
0
5
2
3
0
5
1
6
3
3
0
3
0
4
1
4
1
3
0
11
8
4
1
13
8
13
10
10
7
5
2
9
6
13
10
13
10
8
5
9
6
13
10
12
9
8
5
13
10
13
10
13
10
12
9
400th data
7
4
4
1
5
2
3
0
5
2
4
1
500th data
3
0
6
3
11
8
4
1
13
10
13
10
12
9
9
6
12
9
12
9
600th data
5
2
700th data
3
0
8
5
13
10
10
7
3
0
3
0
4
1
3
0
800th data
3
0
5
0
4
1
3
0
3
0
3
0
5
0
8
5
900th data
3
0
9
6
3
0
4
0
3
0
10
7
3
0
6
1
6
3
4
1
9
6
6
0
3
0
4
1
6
0
6
3
3
0
3
0
3
0
4
0
5
0
3
0
1000th data
3
0
4
1
3
0
3
0
5
2
4
1
4
0
3


In [7]:
year_list = list(range(1993,2020))
result_list = []
temp = []
i = 0

for data in interpolated_list:
    if data[0][2] != year_list[i]:
        i += 1
        result_list.append(np.array(temp))
        temp = []
    temp.append(data)

In [9]:
# import pickle
# for year_idx, _ in enumerate(result_list):
#     fname = f"./data/{year_idx}th_target.pickle"
    
#     with open(fname, "wb") as fp:
#         pickle.dump(result_list[year_idx], fp)